# Exploring Data from Favate et al. 2021

In [90]:
import numpy as np 
import pandas as pd 
import diaux.viz 
import altair as alt 
from altair_saver import save
colors, palette = diaux.viz.altair_style()

In [24]:
# Load the tidied Favate data
data = pd.read_csv('../../data/Favate2021/processed/Favate2021_reads_tidy.csv')

# Set up our own cog class organization
cog_hier = {'Information storage and processing': ['J', 'A', 'K', 'L', 'B'],
            'Cellular processes and signaling': ['D', 'Y', 'V', 'T', 'M', 'N', 'Z', 'W', 'U', 'O'],
            'Metabolism': ['C', 'G', 'E', 'F', 'H', 'I', 'P', 'Q'],
            'Poorly Characterized/Not Assigned': ['R', 'S', 'Not Assigned']}
cog_hier_rev = {}
for k, v in cog_hier.items():
    for letter in v:
        cog_hier_rev[letter] = k

for k, v in cog_hier_rev.items():
    data.loc[data['cog_letter']==k, 'cog_sector'] = v

# For each line and replicate, compute the total fraction of each cog classification
grouped = []
for g, d in data.groupby(['line', 'replicate']):
    total_rnas = d['n_mol_per_cfu'].sum()
    _grouped = d.groupby(['line', 'replicate', 'cog_sector']).sum().reset_index()
    _grouped['fraction'] = _grouped['n_mol_per_cfu'].values / total_rnas 
    grouped.append(_grouped)

agg = pd.concat(grouped, sort=False)


In [25]:
data

,line,repl,target_id,tpm,seqtype,intercept,slope,n_molecules,dilution_1_over,counts_1,...,cells_2_final,cells_3_final,mean_cells_final,n_mol_per_cfu,replicate,gene_name,cog_class,cog_letter,cog_desc,cog_sector
0,Ara-1,rep1,ECB_00001,56.179858,rna,6.409052,0.972421,1.289382e+08,100000,123,...,1.180846e+07,1.068385e+07,1.518231e+07,8.492659,1,thrL,Not Assigned,Not Assigned,Not Assigned,Poorly Characterized/Not Assigned
1,Ara-1,rep1,ECB_00002,31.412891,rna,6.409052,0.972421,7.326080e+07,100000,123,...,1.180846e+07,1.068385e+07,1.518231e+07,4.825406,1,thrA,Amino acid transport and metabolism,E,Homoserine dehydrogenase,Metabolism
2,Ara-1,rep1,ECB_00003,23.354498,rna,6.409052,0.972421,5.491421e+07,100000,123,...,1.180846e+07,1.068385e+07,1.518231e+07,3.616987,1,thrB,Amino acid transport and metabolism,E,Homoserine kinase,Metabolism
3,Ara-1,rep1,ECB_00004,27.902329,rna,6.409052,0.972421,6.528653e+07,100000,123,...,1.180846e+07,1.068385e+07,1.518231e+07,4.300172,1,thrC,Amino acid transport and metabolism,E,Threonine synthase,Metabolism
4,Ara-1,rep1,ECB_00005,10.981519,rna,6.409052,0.972421,2.636419e+07,100000,123,...,1.180846e+07,1.068385e+07,1.518231e+07,1.736507,1,yaaX,Not Assigned,Not Assigned,Not Assigned,Poorly Characterized/Not Assigned
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107401,REL607,rep2,ECB_04274,2.597515,rna,6.701974,0.978606,1.281337e+07,100000,447,...,1.899037e+08,1.859057e+08,1.848396e+08,0.069322,2,creB,Transcription,K,"DNA-binding response regulator, OmpR family, c...",Information storage and processing
107402,REL607,rep2,ECB_04275,1.038380,rna,6.701974,0.978606,5.223732e+06,100000,447,...,1.899037e+08,1.859057e+08,1.848396e+08,0.028261,2,creC,Signal transduction mechanisms,T,Signal transduction histidine kinase,Cellular processes and signaling
107403,REL607,rep2,ECB_04277,22.781589,rna,6.701974,0.978606,1.072788e+08,100000,447,...,1.899037e+08,1.859057e+08,1.848396e+08,0.580389,2,arcA,Transcription,K,"DNA-binding response regulator, OmpR family, c...",Information storage and processing
107404,REL607,rep2,ECB_04278,12.969263,rna,6.701974,0.978606,6.181300e+07,100000,447,...,1.899037e+08,1.859057e+08,1.848396e+08,0.334414,2,yjjY,Not Assigned,Not Assigned,Not Assigned,Poorly Characterized/Not Assigned


In [91]:
charts = []
for rep in [1, 2]:
    bars = alt.Chart(agg[agg['replicate']==rep]).mark_bar().encode(
                x=alt.X(field='n_mol_per_cfu', type='quantitative', title='RNA transcript fraction',
                stack='normalize'),
                detail='line:N',
                order='line:N',
                y=alt.Y(field='line', type='nominal', title='evolved line'),
                color=alt.Color(field='cog_sector', type='nominal', title='COG class')
            )
            
            
    text = alt.Chart(agg[agg['replicate']==rep]).mark_text(dx=-22, color='white').encode(
                x=alt.X(field='n_mol_per_cfu', type='quantitative', title='RNA transcript fraction',
                stack='normalize'),
                y=alt.Y(field='line', type='nominal', title='evolved line'),
                order='line:N',
                # color = alt.Color(field='cog_sector', type='nominal', title='COG_class'),
                text=alt.Text(field='fraction', type='quantitative', format='0.0%')
            )
    charts.append((bars + text))

layer = (charts[0].properties(title='experimental replicate 1') &\
         charts[1].properties(title='experimental replicate 2')
         ).configure_legend(
             labelLimit=1000
         ).properties(
             title='RNA abundance - fraction of molecules per CFU'
         )
save(layer, '/Users/gchure/Desktop/Favate2021_RNA_per_CFU_fraction.pdf')
layer

WARN Channel order is inappropriate for nominal field, which has no inherent order.
WARN Channel order is inappropriate for nominal field, which has no inherent order.
WARN Channel order is inappropriate for nominal field, which has no inherent order.
WARN Channel order is inappropriate for nominal field, which has no inherent order.


alt.VConcatChart(...)

In [115]:
# Look explicitly at the fractional expression for aceA and aceB
ace_df = pd.DataFrame([])
for g, d in data.groupby(['line', 'replicate']):
    total_rna = d['n_mol_per_cfu'].sum()
    ace = d[(d['gene_name']=='aceA') | (d['gene_name']=='aceB')]['n_mol_per_cfu'].sum()
    ace_df = ace_df.append({'line':g[0],
                            'replicate':g[1],
                            'total_aceAB': ace,
                            'fractional_aceAB': ace/total_rna},
                            ignore_index=True)
ace_df.sort_values(by='total_aceAB', inplace=True)
total_ace_points = alt.Chart(ace_df, title='total number of aceAB transcripts per cell').mark_point(size=80, opacity=0.75).encode(
                y=alt.Y(field='line', type='nominal', title='evolved line'),
                x=alt.X(field='total_aceAB', type='quantitative', title='total aceAB transcripts',
                        scale=alt.Scale(type='log')),
                color=alt.Color(field='replicate', type='nominal', title='experimental replicate'))
frac_ace_points = alt.Chart(ace_df, title='fractional abundance of aceAB transcripts per cell').mark_point(size=80, opacity=0.75).encode(
                y=alt.Y(field='line', type='nominal', title='evolved line'),
                x=alt.X(field='fractional_aceAB', type='quantitative', title='aceAB transcript fraction of total',
                        scale=alt.Scale(type='log')),
                color=alt.Color(field='replicate', type='nominal', title='experimental replicate'))
layer = total_ace_points & frac_ace_points
save(layer, '/Users/gchure/Desktop/Favate2021_aceAB_expression.pdf')
layer

alt.VConcatChart(...)